# Memory-Augmented Agent Demo

Goal: show how an AI assistant can remember stable preferences across turns, without leaking everything or storing risky data.

This notebook demonstrates four stages:

1. **Scratchpad build** – summarize the latest user message so the assistant stays on-track in this session.
2. **Answer with memory** – combine:
   - long-term memory (persistent profile in `memory.json`)
   - scratchpad (what we're doing right now)
   - user's question
   and generate a VP-style answer.
3. **Memory write decision** – ask a model if anything NEW from this conversation should enter long-term memory.
4. **Persist** – only if the model explicitly says `SAVE: {"key": ..., "value": ...}`.

This is how you build continuity and personalization you can defend in front of leadership.

In [ ]:
import os, json, textwrap
from pathlib import Path
from openai import OpenAI

# Before running:
# os.environ['OPENAI_API_KEY'] = 'sk-...'

client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
MEMORY_FILE = Path('memory.json')

def load_memory():
    if MEMORY_FILE.exists():
        return json.load(open(MEMORY_FILE, 'r', encoding='utf-8'))
    return {'user_profile': {}, 'last_updated': None}

def save_memory(mem):
    mem['last_updated'] = 'updated_from_notebook'
    json.dump(mem, open(MEMORY_FILE, 'w', encoding='utf-8'), indent=2)

SYSTEM_SCRATCHPAD = """
You are an analyst.
Summarize the user's latest message into a compact scratchpad context that captures:
- What the user is trying to do right now
- Any constraints or style requests
Do NOT add new facts. Be concise.
Return ONLY the summary text.
"""

def build_scratchpad(user_msg: str) -> str:
    resp = client.responses.create(
        model='o4-mini',
        input=[
            {'role': 'system', 'content': SYSTEM_SCRATCHPAD},
            {'role': 'user', 'content': user_msg},
        ],
    )
    if hasattr(resp, 'output_text'):
        return resp.output_text.strip()
    try:
        return resp.choices[0].message.content.strip()
    except Exception:
        if hasattr(resp, 'output'):
            if isinstance(resp.output, list):
                return '\n'.join(str(x) for x in resp.output).strip()
            return str(resp.output)
        return str(resp).strip()

SYSTEM_ANSWER = """
You are an executive briefing assistant.

You will receive:
1. LONG_TERM_MEMORY: stable known preferences about this user
2. SCRATCHPAD: summary of what they just asked
3. USER_QUESTION: the raw question

Your job:
- Answer USER_QUESTION directly
- Respect tone/style preferences in LONG_TERM_MEMORY
- Highlight risk, operational impact, cost if relevant
- Be concise and VP-ready

Output only the final answer for the user.
"""

def answer_with_memory(long_term_memory: dict, scratchpad: str, user_question: str) -> str:
    composite_prompt = textwrap.dedent(f"""
    LONG_TERM_MEMORY:
    {json.dumps(long_term_memory, indent=2)}

    SCRATCHPAD:
    {scratchpad}

    USER_QUESTION:
    {user_question}
    """).strip()

    resp = client.responses.create(
        model='o4-mini',
        input=[
            {'role': 'system', 'content': SYSTEM_ANSWER},
            {'role': 'user', 'content': composite_prompt},
        ],
    )
    if hasattr(resp, 'output_text'):
        return resp.output_text.strip()
    try:
        return resp.choices[0].message.content.strip()
    except Exception:
        if hasattr(resp, 'output'):
            if isinstance(resp.output, list):
                return '\n'.join(str(x) for x in resp.output).strip()
            return str(resp.output)
        return str(resp).strip()

SYSTEM_MEMORY_WRITE = """
You are a memory write policy checker for an AI assistant.

You get:
- The user's latest message
- The assistant's final answer
- The existing LONG_TERM_MEMORY (JSON)

Your job:
1. Decide if there's a NEW stable preference or identity detail that would be valuable long-term.
2. If yes, respond EXACTLY:
SAVE: {"key": "...", "value": "..."}
3. If not, respond EXACTLY:
NOSAVE

Rules:
- Do NOT store secrets, credentials, medical details, or anything too sensitive.
- Store only durable preferences or role context that will clearly help future answers.
"""

def propose_memory_update(long_term_memory: dict, user_msg: str, final_answer: str) -> str:
    composite_prompt = textwrap.dedent(f"""
    USER_MESSAGE:
    {user_msg}

    ASSISTANT_FINAL_ANSWER:
    {final_answer}

    CURRENT_LONG_TERM_MEMORY:
    {json.dumps(long_term_memory, indent=2)}

    Produce memory decision now.
    """).strip()

    resp = client.responses.create(
        model='o4-mini',
        input=[
            {'role': 'system', 'content': SYSTEM_MEMORY_WRITE},
            {'role': 'user', 'content': composite_prompt},
        ],
    )
    if hasattr(resp, 'output_text'):
        return resp.output_text.strip()
    try:
        return resp.choices[0].message.content.strip()
    except Exception:
        if hasattr(resp, 'output'):
            if isinstance(resp.output, list):
                return '\n'.join(str(x) for x in resp.output).strip()
            return str(resp.output)
        return str(resp).strip()

def apply_memory_update(mem: dict, decision: str) -> (dict, str):
    decision = decision.strip()
    if decision.startswith('NOSAVE'):
        return mem, 'No persistent memory update.'

    if decision.startswith('SAVE:'):
        payload = decision[len('SAVE:'):].strip()
        try:
            data = json.loads(payload)
            key = data.get('key')
            value = data.get('value')
        except Exception:
            return mem, f'Memory decision malformed: {decision}'

        if not key or not value:
            return mem, f'Memory decision missing key/value: {decision}'

        if 'user_profile' not in mem or not isinstance(mem['user_profile'], dict):
            mem['user_profile'] = {}
        mem['user_profile'][key] = value
        save_memory(mem)
        return mem, f'Stored memory: {key} = {value}'

    return mem, f'Unrecognized memory decision: {decision}'


### Demo conversation turn
Pretend the user says:

> "I'm visiting Hyderabad again next week to meet execs. Give me a short briefing with risks and travel tips. I prefer VP tone and no fluff."

We'll walk through:
1. load existing memory.json
2. build scratchpad
3. answer_with_memory
4. propose_memory_update
5. apply_memory_update


In [ ]:
user_msg = (
    "I'm visiting Hyderabad again next week to meet execs. "
    "Give me a short briefing with risks and travel tips. "
    "I prefer VP tone and no fluff."
)

mem = load_memory()
scratch = build_scratchpad(user_msg)
final_answer = answer_with_memory(mem, scratch, user_msg)
decision = propose_memory_update(mem, user_msg, final_answer)
updated_mem, note = apply_memory_update(mem, decision)

print('===== SCRATCHPAD =====')
print(scratch)
print('\n===== FINAL ANSWER =====')
print(final_answer)
print('\n===== MEMORY DECISION =====')
print(decision)
print('\n===== MEMORY UPDATE NOTE =====')
print(note)
print('\n===== UPDATED MEMORY =====')
print(json.dumps(updated_mem, indent=2))


### Why this matters
This pipeline:

- Uses **long-term memory** (persistent JSON) to maintain user profile.
- Builds a **short-term scratchpad** so the model stays on track in the current turn without full chat history.
- Uses a **memory write policy step** to decide if anything new should be remembered and logs that decision.

That last part is critical for safety and compliance:
you can review every proposed memory before it gets saved, or even require a human approval step in regulated environments.

This is how you move from a stateless chatbot to a durable AI chief of staff with audit trails.